In [199]:
import pandas as pd
from bs4 import BeautifulSoup
from typing import List, Dict, Tuple
import requests
import json

In [200]:
PATH_CRAN_CATEGORIES="https://cran.rstudio.com/web/views/"
html_content = requests.get(PATH_CRAN_CATEGORIES).content

In [201]:
def get_categories(html: str="", PATH_CATEGORIES: str=PATH_CRAN_CATEGORIES)-> Dict[str, str]:
    d_package = {}
    soup = BeautifulSoup(html, 'html.parser')
    for pack in soup.select("table > tr"):
        keys = pack.select("td:nth-of-type(2)")[0].text
        values = PATH_CATEGORIES+pack.select("td:nth-of-type(1)")[0].text+".html"
        d_package[keys] = values
    return d_package

In [202]:
d_cat_link = get_categories(html_content, PATH_CATEGORIES=PATH_CRAN_CATEGORIES)

In [203]:
def get_category_description(packages: Dict[str, str])-> Tuple[Dict[str, List[str]], Dict[str, List[str]]]:
    d_category_pack = {}
    d_pack_cat_descri = {}
    list_packages = []
    for key, value in packages.items():
        html_category = requests.get(value).content
        soup = BeautifulSoup(html_category, 'html.parser')
        # List of packages are just after h3 title CRAN packages
        table = soup.find("h3", string="CRAN packages").findNext("table")
        pack_name = [val.text for val in table.select("a")]
        list_packages += pack_name
        d_category_pack[key] = pack_name

    #Remove duplicate using set
    for p in list(set(list_packages)):
        html_package = requests.get("https://cran.rstudio.com/web/packages/"+p+"/index.html").content
        soup = BeautifulSoup(html_package, 'html.parser')
        description = soup.find("h2")
        if(description!= None):
            #Getting category
            views = soup.find_all("a")
            l_c = []
            for sub_a in views:
                if("../../views/" in str(sub_a)):
                    l_c.append(sub_a.text)

            #Adding both categories and description
            d_pack_cat_descri[p] = {"categories": l_c, "description": description.findNext("p").text}
        else:
            print("Package {} has expired.".format(p))
    return d_category_pack, d_pack_cat_descri

In [204]:
d_category_pack, d_pack_cat_descri = get_category_description(d_cat_link)

Package lmec has expired.
Package hiPOD has expired.
Package Vdgraph has expired.
Package R0 has expired.
Package glrt has expired.
Package RCassandra has expired.
Package abcrf has expired.
Package AdjBQR has expired.
Package SampleSizeMeans has expired.
Package bbemkr has expired.
Package smds has expired.
Package SpaTimeClus has expired.
Package TScompare has expired.
Package EcoHydRology has expired.
Package cpd has expired.
Package Stem has expired.
Package astrolibR has expired.
Package atmcmc has expired.
Package constrainedKriging has expired.
Package BBMM has expired.
Package kcirt has expired.
Package csodata has expired.
Package fishmove has expired.
Package gradDescent has expired.
Package crrstep has expired.
Package sparsebn has expired.
Package eyetracking has expired.
Package cutoffR has expired.
Package aurelius has expired.
Package osmar has expired.
Package qtlDesign has expired.
Package CompRandFld has expired.
Package ssvd has expired.
Package predmixcor has expire

In [207]:
with open("R_Cran_views.json", "w") as outfile:
    json.dump(d_cat_link, outfile)

with open("R_Cran_Categories.json", "w") as outfile:
    json.dump(d_category_pack, outfile)

with open("R_Cran_Packages.json", "w") as outfile:
    json.dump(d_pack_cat_descri, outfile)